# Image Classification with ResNet50

This Python script demonstrates how to perform image classification using a pre-trained ResNet50 model. The script loads the ResNet50 model, fine-tunes it on a custom dataset of image classes, and then uses it to classify a given input image. Below are the steps to use this script:

## How to Use

1. **Install Required Libraries:**
   Before running the script, make sure you have the necessary libraries installed. You can install them using the following commands:

   ```bash
   pip install tensorflow
   pip install pillow
   ```

2. **Directory Structure:**
   - Ensure that you have a directory containing training images organized into subdirectories for each class. In the script, the `train_dir` variable should point to the directory containing your training data.
   - The `classes` list should contain the names of the classes corresponding to the subdirectories.

3. **Load Pre-trained ResNet50 Model:**
   - The script loads the pre-trained ResNet50 model (excluding the top classification layer).
   - It then adds custom classification layers on top of the base model.

4. **Compile the Model:**
   - The model is compiled with an optimizer, loss function, and metrics for training.

5. **Load and Preprocess Data:**
   - The script loads and preprocesses the training data from the specified directory.
   - Images are resized to 224x224 pixels and preprocessed according to the ResNet50 requirements.

6. **Train the Model:**
   - The model is trained on the preprocessed training data for a specified number of epochs.

7. **Classify Input Image:**
   - You can specify the path to an input image using the `input_image_path` variable.
   - The script loads and preprocesses the input image, and then uses the trained model to predict the class of the input image.

8. **View the Prediction:**
   - The script prints the predicted class label for the input image.

## Dependencies

This script relies on the following Python libraries:

- `os`: Used for directory and file operations.
- `numpy`: For numerical operations.
- `tensorflow`: TensorFlow library for machine learning.
- `tensorflow.keras`: The Keras API within TensorFlow for building and training deep learning models.
- `PIL`: Python Imaging Library for image processing.

Feel free to modify and extend the script to suit your specific image classification needs or integrate it into a larger machine learning pipeline.

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from PIL import Image

# Directory containing training images
train_dir = 'Priya_Goyal/'

# List of image classes
classes = ['cat','dog','chair','rose']

# Load pre-trained ResNet50 model (excluding top classification layer)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
predictions = tf.keras.layers.Dense(len(classes), activation='softmax')(x)

# Create the final model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Load and preprocess data
X_train = []
y_train = []

for idx, class_name in enumerate(classes):
    class_dir = os.path.join(train_dir, class_name)
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            X_train.append(img_array)
            y_train.append(idx)

X_train = np.vstack(X_train)
y_train = np.array(y_train)

# Train the model
model.fit(X_train, y_train, epochs=10)

def prediction(input_image_path):
    # Input image to classify
    input_image = image.load_img(input_image_path, target_size=(224, 224))
    input_image_array = image.img_to_array(input_image)
    input_image_array = np.expand_dims(input_image_array, axis=0)
    input_image_array = preprocess_input(input_image_array)

    # Classify the input image
    predictions = model.predict(input_image_array)
    predicted_class_idx = np.argmax(predictions)
    predicted_class = classes[predicted_class_idx]
    print("Predicted class:", predicted_class)
    
    


Epoch 1/10
1/1 [==============================] - 12s 12s/step - loss: 1.3693 - accuracy: 0.4286
Epoch 2/10
1/1 [==============================] - 6s 6s/step - loss: 0.1465 - accuracy: 0.9524
Epoch 3/10
1/1 [==============================] - 6s 6s/step - loss: 0.0063 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 5s 5s/step - loss: 6.7202e-04 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 5s 5s/step - loss: 1.7978e-04 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 6s 6s/step - loss: 6.9273e-05 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 5s 5s/step - loss: 3.2220e-05 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 6s 6s/step - loss: 1.6994e-05 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 6s 6s/step - loss: 9.6667e-06 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 6s 6s/step - loss: 5.9091e-06 - accuracy: 1.0000


In [2]:
# Testing model
input_image_path = "cat6.jpg"
prediction(input_image_path)

1/1 [==============================] - 2s 2s/step
Predicted class: cat


In [3]:
# Testing model
input_image_path = "rose7.jpeg"
prediction(input_image_path)

1/1 [==============================] - 1s 517ms/step
Predicted class: rose


In [4]:
# Testing model
input_image_path = "dog6.jpeg"
prediction(input_image_path)

1/1 [==============================] - 0s 377ms/step
Predicted class: dog


In [7]:
# Testing model
input_image_path = "chair7.jpg"
prediction(input_image_path)

1/1 [==============================] - 0s 435ms/step
Predicted class: chair
